In [43]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
import re

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [268]:
df2011 = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/census2011/2011_Full_Data.csv')
df2020 = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/census2011/2020_Full_Data.csv')
workers = pd.read_csv('/Users/sid/Downloads/category-wise-workers.csv')
lgd_full = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/census2011/lgd2024.csv')


In [269]:
df2020 = df2020[[
    'State_code',
    'State',
    'District_code',
    'District',
    'Subdistrict_code',
    'Subdistrict',
    'Village_code',
    'Village',
    'GP_code',
    'GP',
    'total_hhd',
    'total_population',
    'male_population',
    'female_population',
    'total_hhd_engaged_in_farm_activities',
    'total_hhd_having_bpl_cards',
    'availability_of_primary_school',
    'availability_of_middle_school',
    'availability_of_high_school',
    'availability_of_ssc_school',
    'availability_of_govt_degree_college',
    'is_vocational_edu_centre_available',
    'availability_of_phc_chc',
    'availability_of_jan_aushadhi_kendra',
    'availability_of_mother_child_health_facilities',
    'is_veterinary_hospital_available',
    'is_aanganwadi_centre_available',
    'is_bank_available',
    'is_post_office_available',
    'availability_of_public_transport',
    'availability_of_railway_station',
    'is_village_connected_to_all_weather_road',
    'availability_of_market',
    'nearest_urban_proximity',
    'Village',
    'stcode11',
    'dtcode11',
    'sdtcode11',
    'vilcode11',
    'state',
    'district',
    'subdistrict',
    'vilname11',
    'vil_code',
    'dist_lgd',
    'state_lgd',
    'gpcode',
    'gp',
    'subdt_lgd',
    'Block',
    'block_lgd',
    'village_area_sqkm',
    'SubDistrict Area',
    'avg_ntl',
    'ntl_pa',
    'ntl_pc'
]]

df2020 = df2020.rename(columns={
                                           'total_population':'population',
                                  'male_population':'male_pop','female_population':'female_pop',
                                  'total_hhd_engaged_in_farm_activities':'hh_in_farm','total_hhd_having_bpl_cards':'bpl_cards_hh',
                                  'availability_of_primary_school':'p_school','availability_of_middle_school':'m_school',
                                  'availability_of_high_school':'h_school','availability_of_ssc_school':'ssc_school',
                                  'availability_of_govt_degree_college':'arts_and_science_degree_college',
                                  'is_vocational_edu_centre_available':'iti',
                                  'availability_of_phc_chc':'phc',
                                  'availability_of_mother_child_health_facilities':'mcwc','availability_of_jan_aushadhi_kendra':'dispensary',
                                  'is_veterinary_hospital_available':'veterinary_hospital','is_post_office_available':'post_office',
                                  'availability_of_public_transport':'bus',
                                  'availability_of_railway_station':'railway',
                                  'is_village_connected_to_all_weather_road':'all_weather_road',
                                  'is_bank_available':'cooperative_bank',
                                  'availability_of_market':'mandis',
                                  'is_aanganwadi_centre_available':'aanganwadi',                                  
                                  'nearest_urban_proximity':'nearest_town_distance'
                                  })

df2011['sc_pop_share']=df2011['sc_pop']/df2011['population']
df2011['st_pop_share']=df2011['st_pop']/df2011['population']
df2020['sc_pop_share'] = None
df2020['st_pop_share'] = None
df2011['year']=2011
df2020['year']=2020


# Handle duplicate column names by keeping the first occurrence
df2020 = df2020.loc[:, ~df2020.columns.duplicated()]

# Check for unique column names
if not df2020.columns.is_unique or not df2011.columns.is_unique:
    raise ValueError("One of the DataFrames has non-unique column names.")

# Find the common columns
common_columns = df2020.columns.intersection(df2011.columns)

# Subset both DataFrames to include only these common columns
df2020_filt = df2020[common_columns]
df2011_filt = df2011[common_columns]



In [270]:
workers = workers.rename(columns={
                                    'state_name':'state',
                                    'district_name':'district',
                                    'gp_name':'gp',
                                    'block_name':'Block',
                                    })

workers=workers.drop(columns={'id'},axis=1)

In [271]:
# Define a function to clean text
def clean_text(text):
    if isinstance(text, str):  # Check if it's a string
        # Remove dots, commas, hyphens
        text = re.sub(r'[.,-]', '', text)
        # Remove trailing and leading whitespace
        text = text.strip()
        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
    return text

# Clean columns 'district', 'Block', and 'gp'
workers['state'] = workers['state'].str.title()
# workers['district'] = workers['district'].apply(clean_text)
# workers['Block'] = workers['Block'].apply(clean_text)
# workers['gp'] = workers['gp'].astype(str).apply(clean_text)

df2020_filt['state'] = df2020_filt['state'].str.title()
# df2020['district'] = df2020['district'].apply(clean_text)
# df2020['Block'] = df2020['Block'].apply(clean_text)
# df2020['gp'] = df2020['gp'].astype(str).apply(clean_text)

df2011_filt['state'] = df2011_filt['state'].str.title()
# df2011['district'] = df2011['district'].apply(clean_text)
# df2011['Block'] = df2011['Block'].apply(clean_text)
# df2011['gp'] = df2011['gp'].astype(str).apply(clean_text)

lgd_full['state'] = lgd_full['state'].str.title()
# lgd_full['district'] = lgd_full['district'].apply(clean_text)
# lgd_full['Block'] = workers['Block'].apply(clean_text)
# lgd_full['gp'] = lgd_full['gp'].astype(str).apply(clean_text)

In [272]:
state_to_zone = {'Andhra Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa',
                 'Gujarat', 'Haryana', 'Jharkhand', 'Karnataka', 'Kerala',
                 'Madhya Pradesh', 'Maharashtra', 'Odisha', 'Punjab', 'Rajasthan',
                 'Tamil Nadu', 'Telangana', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'}

# Define a function to filter the DataFrame
def filter_states(df, state_set):
    return df[df['state'].isin(state_set)]

# Apply the filter to each DataFrame
data11 = filter_states(df2011_filt, state_to_zone)
data20 = filter_states(df2020_filt, state_to_zone)
workers = filter_states(workers, state_to_zone)
lgd_full = filter_states(lgd_full, state_to_zone)

In [273]:
data11

,Village,population,male_pop,female_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,nearest_town_distance,stcode11,dtcode11,sdtcode11,vilcode11,state,district,subdistrict,vilname11,vil_code,dist_lgd,state_lgd,gpcode,gp,subdt_lgd,Block,block_lgd,village_area_sqkm,SubDistrict Area,avg_ntl,ntl_pa,sc_pop_share,st_pop_share,year
0,KARANJI,2232,1104.0,1128.0,1,1,1,0,0,0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,52.0,36,532,NaN,568945.0,Telangana,Adilabad,BHEEMPUR,Karanji (T),568945,501,36,195687,KARANJI (T),6223,Bheempur,7189,22.147866,222.803432,19.455715,0.878446,0.115591,0.356183,2011
1,GUBIDI,509,254.0,255.0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,51.0,36,532,NaN,568946.0,Telangana,Adilabad,BHEEMPUR,Guledi,568946,501,36,288862,Gubdi,6223,Bheempur,7189,8.616328,222.803432,10.397142,1.206679,0.035363,0.229862,2011
2,GOMUTRI,1102,551.0,551.0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,40.0,36,532,NaN,568947.0,Telangana,Adilabad,BHEEMPUR,Gomutri,568947,501,36,195683,GOMUTRI,6223,Bheempur,7189,12.704872,222.803432,11.481429,0.903703,0.070780,0.315789,2011
3,ANTARGAON,875,445.0,430.0,1,1,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0,0,1,0,37.0,36,532,NaN,568948.0,Telangana,Adilabad,BHEEMPUR,Antargaon,568948,501,36,195675,ANTARGAON,6223,Bheempur,7189,6.287569,222.803432,8.255713,1.313022,0.109714,0.147429,2011
4,ARLI TARAF,2833,1428.0,1405.0,1,1,1,0,0,0,0.0,0.0,0.0,0.0,1,1,0,0,0,1,0,32.0,36,532,NaN,568949.0,Telangana,Adilabad,BHEEMPUR,Arli (T),568949,501,36,195676,ARLI (T),6223,Bheempur,7189,33.909260,222.803432,30.801426,0.908349,0.052241,0.378750,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498459,AAJANDA,728,385.0,343.0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,45.0,8,131,00700,108874.0,Rajasthan,Pratapgarh,Arnod,Ajanda,108874,629,8,262185,Bhat Bhamriya,700,DALOT,7466,2.448718,786.360812,6.937143,2.832969,0.000000,1.000000,2011
498460,BHATBHAMRIYA,1551,784.0,767.0,1,1,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0,0,1,0,45.0,8,131,00700,108875.0,Rajasthan,Pratapgarh,Arnod,Bhat Bhamriya,108875,629,8,262185,Bhat Bhamriya,700,DALOT,7466,6.783740,786.360812,17.815716,2.626238,0.000645,0.992908,2011
498461,LUHARIKHARI,555,289.0,266.0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,35.0,8,131,00700,108878.0,Rajasthan,Pratapgarh,Arnod,,108878,629,8,295222,Luharkhali,700,DALOT,7466,12.098066,786.360812,37.167140,3.072156,0.000000,1.000000,2011
498462,AMBIRAMA,1472,728.0,744.0,1,1,0,0,0,0,1.0,0.0,0.0,0.0,1,0,0,0,0,0,0,30.0,8,131,00700,108879.0,Rajasthan,Pratapgarh,Arnod,Ambeerama,108879,629,8,36396,AMBEE RAMA,700,DALOT,7466,16.400206,786.360812,45.257140,2.759547,0.043478,0.870924,2011


## LGD Cleaning

In [274]:
# # Specify the path to your Excel file
# file_path = '/Users/sid/Downloads/LGD_Villages2024.xlsx'

# # Read the first sheet with headers
# first_sheet = pd.read_excel(file_path, sheet_name=0)

# # Get the column names from the first sheet
# column_names = first_sheet.columns

# # Read all sheets without headers
# all_sheets = pd.read_excel(file_path, sheet_name=None, header=None)

# # Remove the first sheet from the dictionary
# del all_sheets[list(all_sheets.keys())[0]]

# # Rename the columns of the remaining sheets to match the first sheet
# for sheet_name in all_sheets:
#     all_sheets[sheet_name].columns = column_names

# # Concatenate the remaining sheets
# remaining_sheets = pd.concat(all_sheets.values(), ignore_index=True)

# # Concatenate the first sheet (with headers) and the remaining sheets (with renamed headers)
# vill_list = pd.concat([first_sheet, remaining_sheets], ignore_index=True)

In [275]:
# # Specify the path to your Excel file
# file_path = '/Users/sid/Downloads/subdistrictVillageBlockGpsMapping2024.xlsx'

# # Read the first sheet with headers
# first_sheet = pd.read_excel(file_path, sheet_name=0)

# # Get the column names from the first sheet
# column_names = first_sheet.columns

# # Read all sheets without headers
# all_sheets = pd.read_excel(file_path, sheet_name=None, header=None)

# # Remove the first sheet from the dictionary
# del all_sheets[list(all_sheets.keys())[0]]

# # Rename the columns of the remaining sheets to match the first sheet
# for sheet_name in all_sheets:
#     all_sheets[sheet_name].columns = column_names

# # Concatenate the remaining sheets
# remaining_sheets = pd.concat(all_sheets.values(), ignore_index=True)

# # Concatenate the first sheet (with headers) and the remaining sheets (with renamed headers)
# block_mapping = pd.concat([first_sheet, remaining_sheets], ignore_index=True)

In [276]:
# lgd_full = pd.merge(vill_list,block_mapping,on='Village Code',how='inner')

# lgd_full = lgd_full.drop(columns={'State Name ','District Name ','Village Version','Village Name.1','S.No._x',
#                                   'Village Status','S.No._y', 'State Code_y','State Census  2011 Code','District Code_y',
#                                   'Sub-District Code','Sub-District Name', 'District Census 2011 Code', '   Village Name   ',
#                                   'Village Census 2011 Code', 'Localbody Census 2011 Code', 'District Code of Development Block',
#                                   'District Name of Development Block'
#                                   })
# lgd_full = lgd_full.rename(columns={'State Code_x':'state_lgd',
#                                     'District Code_x':'dist_lgd',
#                                     'Village Code':'vil_code',
#                                     'Village Name':'Village',
#                                     'State Name':'state',
#                                     'District Name':'district',
#                                     'Subdistrict Code ':'district_code',
#                                     'Subdistrict Name':'subdt_lgd',
#                                     'Localbody Code':'gpcode',
#                                     'localbody Name':'gp',
#                                     'Development Block Code':'block_lgd',
#                                     'Development Block Name':'Block',
#                                     'Census 2001 Code':'census2001',
#                                     'Census 2011 Code':'census2011',
#                                     'Subdistrict Census 2011 Code':'census2011_sdt'
#                                     })
# len(lgd_full)

In [277]:
# lgd_full.to_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/census2011/lgd2024.csv')


## LGD - MGNREGA Merge

### Block Level

In [278]:
workers['year'] = workers['year'].str.split('-').str[0]

workers20 = workers[workers['year']=="2019"]
workers14 = workers[workers['year']=="2014"]

In [279]:
lgd_match_clean = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/MNREGA/lgd_merge.csv')
merge1_work_match20 = pd.merge (workers20,lgd_match_clean,on=['state','district','Block'],how='left')
merge1_work_match14 = pd.merge (workers14,lgd_match_clean,on=['state','district','Block'],how='left')


In [280]:
# Define a function to replace values
def replace_values(row):
    if pd.notna(row['state_m']) and row['state'] != row['state_m']:
        row['state'] = row['state_m']
    if pd.notna(row['district_m']) and row['district'] != row['district_m']:
        row['district'] = row['district_m']
    if pd.notna(row['Block_m']) and row['Block'] != row['Block_m']:
        row['Block'] = row['Block_m']
    return row

# Apply the function to each row
merge1_work_match20 = merge1_work_match20.apply(replace_values,axis=1)

# Drop the temporary columns
merge1_work_match20 = merge1_work_match20.drop(columns=['state_m', 'district_m', 'Block_m',
                                                    'State Code', 'District Code',
                                                    'Subdistrict Code', 'subdistrict', 
                                                    'Development Block Code'
                                                    ])
# Apply the function to each row
merge1_work_match14 = merge1_work_match14.apply(replace_values,axis=1)

# Drop the temporary columns
merge1_work_match14 = merge1_work_match14.drop(columns=['state_m', 'district_m', 'Block_m',
                                                    'State Code', 'District Code',
                                                    'Subdistrict Code', 'subdistrict', 
                                                    'Development Block Code'
                                                    ])

In [315]:
merge1_work_match20['num']=1
merge1_work_match14['num']=1

blk_20 = merge1_work_match20.groupby(['state','district','Block']).sum().reset_index()
blk_14 = merge1_work_match14.groupby(['state','district','Block']).sum().reset_index()

blk_20

,state,district,Block,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,count,num
0,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,12545.0,12545.0,947.0,11962.0,2282.0,15191.0,7743.0,8413.0,827.0,10771.0,1874.0,13472.0,6893.0,22,22
1,Andhra Pradesh,Alluri Sitharama Raju,Ananthagiri,12986.0,12986.0,430.0,24946.0,339.0,25715.0,12652.0,10828.0,399.0,23699.0,308.0,24406.0,11953.0,24,24
2,Andhra Pradesh,Alluri Sitharama Raju,Araku Valley,15844.0,15844.0,136.0,28585.0,159.0,28880.0,14649.0,13114.0,118.0,27074.0,144.0,27336.0,13809.0,14,14
3,Andhra Pradesh,Alluri Sitharama Raju,Chintapalle,15715.0,15715.0,206.0,31153.0,282.0,31641.0,15811.0,13178.0,177.0,28031.0,206.0,28414.0,14069.0,17,17
4,Andhra Pradesh,Alluri Sitharama Raju,Chintoor,11875.0,11875.0,226.0,15158.0,683.0,16067.0,8500.0,7899.0,163.0,13311.0,467.0,13941.0,7375.0,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6016,West Bengal,Uttar Dinajpur,Islampur,59407.0,58750.0,15219.0,1543.0,94811.0,111573.0,52733.0,31042.0,5749.0,505.0,43225.0,49479.0,25048.0,13,13
6017,West Bengal,Uttar Dinajpur,Itahar,53972.0,53970.0,27937.0,5971.0,63999.0,97907.0,45325.0,26154.0,11136.0,1844.0,32383.0,45363.0,22012.0,12,12
6018,West Bengal,Uttar Dinajpur,Kaliaganj,48413.0,48413.0,65727.0,4984.0,30295.0,101006.0,46568.0,25008.0,20303.0,1206.0,8162.0,29671.0,16363.0,8,8
6019,West Bengal,Uttar Dinajpur,Karandighi,61075.0,61068.0,30117.0,5409.0,82753.0,118279.0,55258.0,30108.0,11366.0,1957.0,39546.0,52869.0,25522.0,13,13


In [316]:
lgd = lgd_full[['state_lgd', 'dist_lgd','state', 'district', 'district_code',
                'block_lgd', 'Block']].drop_duplicates()

merge2_work_lgd20 = pd.merge(blk_20,lgd,on=['state','district','Block'],how='inner').drop_duplicates(subset=['state','district','Block'])
merge2_work_lgd14 = pd.merge(blk_14,lgd,on=['state','district','Block'],how='inner').drop_duplicates(subset=['state','district','Block'])



print(len(merge2_work_lgd20))
print(len(merge2_work_lgd14))


5547
5153


## MNREGA - MA Merge

In [402]:
# merge2_work_lgd20
# merge2_work_lgd14

loc_list20 = data20[['state_lgd', 'dist_lgd','state', 'district',
                    'block_lgd',]].drop_duplicates()

loc_list11 = data11[['state_lgd', 'dist_lgd','state', 'district',
                    'block_lgd']].drop_duplicates()

print(len(loc_list20))
print(len(loc_list11))


6278
5701


In [406]:
# # Columns to drop
# drop_columns = ['vil_code', 'gpcode', 'subdt_lgd', 'State_code', 'State', 'District_code', 
#                 'District', 'Subdistrict_code', 'Subdistrict', 'Village_code', 'Village', 'GP_code', 'GP']

# # Drop specified columns
# df = data20.drop(columns=drop_columns)
df20 = data20.copy()
df20['count']=1
df20 = df20.drop(columns={'sdtcode11','Village','stcode11', 'dtcode11', 'vilcode11',
       'subdistrict', 'vilname11', 'vil_code', 'gpcode', 'gp','Block'})

df11 = data11.copy()
df11['count']=1
df11 = df11.drop(columns={'sdtcode11','Village','stcode11', 'dtcode11', 'vilcode11',
       'subdistrict', 'vilname11', 'vil_code', 'gpcode', 'gp','Block'})

# List of columns to group by
groupby_columns = ['year','state', 'state_lgd', 'district', 'dist_lgd', 'block_lgd']

# Columns to mean
mean_columns = ['SubDistrict Area','subdt_lgd']

# Columns to sum (all columns except the groupby columns and the mean columns)
sum_columns = [col for col in df20.columns if col not in groupby_columns + mean_columns]

# Group by and aggregate
df20_agg = df20.groupby(groupby_columns).agg({**{col: 'sum' for col in sum_columns},
                                                 **{col: 'mean' for col in mean_columns}}).reset_index()

df11_agg = df11.groupby(groupby_columns).agg({**{col: 'sum' for col in sum_columns},
                                                 **{col: 'mean' for col in mean_columns}}).reset_index()

print(df20_agg.shape)
print(df11_agg.shape)

(6278, 35)
(5701, 35)


In [325]:
merge3_2020_data = pd.merge(df20_agg,merge2_work_lgd20,on=['block_lgd'],how='inner')
merge3_2011_data = pd.merge(df11_agg,merge2_work_lgd14,on='block_lgd',how='inner')

merge3_2020_data = merge3_2020_data.drop(columns={'state_y', 'district_y',
                                                  'state_lgd_y', 'dist_lgd_y'})
merge3_2020_data = merge3_2020_data.rename(columns={'state_x':'state', 'district_x':'district',
                                                  'state_lgd_x':'state_lgd',
                                                  'dist_lgd_x':'dist_lgd'})

merge3_2011_data = merge3_2011_data.drop(columns={'state_y', 'district_y',
                                                  'state_lgd_y', 'dist_lgd_y'})
merge3_2011_data = merge3_2011_data.rename(columns={'state_x':'state', 'district_x':'district',
                                                  'state_lgd_x':'state_lgd',
                                                  'dist_lgd_x':'dist_lgd'})

print(merge3_2020_data.shape)
print(merge3_2011_data.shape)

(5519, 51)
(4617, 51)


In [326]:
finalmerge = pd.concat([merge3_2020_data,merge3_2011_data])

# Your state_to_zone dictionary
state_to_zone = {
    'Andhra Pradesh': 'A',
    'Assam': 'B',
    'Bihar': 'B',
    'Chhattisgarh': 'B',
    'Goa': 'A',
    'Gujarat': 'A',
    'Haryana': 'B',
    'Jharkhand': 'B',
    'Karnataka': 'A',
    'Kerala': 'A',
    'Madhya Pradesh': 'B',
    'Maharashtra': 'A',
    'Odisha': 'B',
    'Punjab': 'B',
    'Rajasthan': 'B',
    'Tamil Nadu': 'A',
    'Telangana': 'A',
    'Uttar Pradesh': 'B',
    'Uttarakhand': 'B',
    'West Bengal': 'B'
}

# Filter the DataFrame
finalmerge['Region'] = finalmerge['state'].map(state_to_zone)

# Create the 'period' column based on the 'year' column
finalmerge['period'] = np.where(finalmerge['year'] == 2011, 0, 
                                    np.where(finalmerge['year'] == 2020, 1, np.nan))


finalmerge['unique_id'] = finalmerge['state'] + '_' + finalmerge['district'] + '_' + finalmerge['Block']

# If you want to drop rows where 'year' is not 2011 or 2020
finalmerge.dropna(subset=['period'], inplace=True)


In [329]:


pivot_df = finalmerge.pivot(index=['state','district','block_lgd'], columns='year', values=['mcwc','num']).reset_index()

# Rename columns for clarity
pivot_df.columns = ['state','district','block_lgd', 'mcwc_2011', 'mcwc_2020','num_2011','num_2020']

# Create the 'treated' column based on the comparison of mcwc values
pivot_df['treated'] = np.where(pivot_df['mcwc_2011'] > pivot_df['mcwc_2020'], 0, 1)
pivot_df['mcwc_dens_2011']=pivot_df['mcwc_2011']/pivot_df['num_2011']
pivot_df['mcwc_dens_2020']=pivot_df['mcwc_2020']/pivot_df['num_2020']
pivot_df['intensity']=pivot_df['mcwc_dens_2020']-pivot_df['mcwc_dens_2011']

# Adjust intensity based on the conditions
pivot_df['intensity'] = pivot_df.apply(
    lambda row: 0 if row['intensity'] < 0 or row['mcwc_2011'] == row['mcwc_2020'] else row['intensity'], 
    axis=1
)



# Merge the treated column back to the original DataFrame
finalmerge2 = pd.merge(finalmerge, pivot_df[['state','district','block_lgd', 'treated',
                                             'intensity','mcwc_dens_2020','mcwc_dens_2011'
                                             ]]
                                , on=['state','district','block_lgd'], how='left')

# Drop duplicates to avoid issues in the merge step
finalmerge2 = finalmerge2.drop_duplicates(subset=['block_lgd', 'year'])

finalmerge2


,year,state,state_lgd,district,dist_lgd,block_lgd,population,male_pop,female_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,nearest_town_distance,village_area_sqkm,avg_ntl,ntl_pa,sc_pop_share,st_pop_share,SubDistrict Area,subdt_lgd,Block,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,count,num,district_code,Region,period,unique_id,treated,intensity,mcwc_dens_2020,mcwc_dens_2011
0,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4865.0,31713.0,15893.0,15801.0,45,9,6,2,1,2,5.0,2.0,21.0,5.0,47,1,15,16,0,30,2,157.035025,485.686671,1265.577775,213.397548,0,0,547.778669,4887.0,Addateegala,12545.0,12545.0,947.0,11962.0,2282.0,15191.0,7743.0,8413.0,827.0,10771.0,1874.0,13472.0,6893.0,22,22,4887,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateegala,1,0.954545,0.954545,0.000000
1,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4874.0,28032.0,13257.0,14730.0,33,8,5,2,1,0,6.0,0.0,15.0,2.0,34,1,15,9,0,20,1,189.385115,377.227968,1172.645552,138.332336,0,0,377.227968,4885.0,Devipatnam,9314.0,9314.0,708.0,8234.0,2502.0,11444.0,5620.0,6366.0,606.0,7292.0,2042.0,9940.0,4868.0,14,14,4885,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Devipatnam,1,1.071429,1.071429,0.000000
2,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4876.0,23219.0,11002.0,12115.0,28,11,5,3,0,1,13.0,1.0,18.0,7.0,30,5,13,8,0,22,1,201.777252,352.122052,913.918889,134.681193,0,0,378.724208,4894.0,Gangavaram,10455.0,10455.0,1058.0,10001.0,3617.0,14676.0,7745.0,7339.0,978.0,9279.0,3379.0,13636.0,7248.0,17,17,4894,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Gangavaram,1,1.058824,1.058824,0.000000
3,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4895.0,10846.0,5463.0,5382.0,19,5,2,2,0,1,4.0,0.0,7.0,3.0,20,1,3,4,0,8,1,33.175603,91.935201,264.876667,86.765174,0,0,1079.650769,4884.0,Maredumilli,5988.0,5988.0,39.0,7513.0,45.0,7597.0,3871.0,3983.0,22.0,6309.0,33.0,6364.0,3261.0,12,12,4884,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Maredumilli,1,0.583333,0.583333,0.000000
4,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4905.0,42730.0,21429.0,21191.0,50,14,8,3,3,2,7.0,7.0,22.0,6.0,52,3,17,10,0,39,3,237.321362,297.879966,959.093192,217.999253,0,0,528.940158,4888.0,Rajavommangi,12551.0,12551.0,285.0,11182.0,5317.0,16784.0,8657.0,8945.0,256.0,10230.0,4991.0,15477.0,8023.0,19,19,4888,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Rajavommangi,1,1.157895,1.157895,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10131,2011,West Bengal,19.0,Purulia,321.0,3059.0,285238.0,146748.0,138490.0,218,76,60,36,22,24,12.0,8.0,8.0,16.0,196,22,50,20,28,108,28,0.000000,662.198467,2389.560003,883.210816,29.020364,32.941493,338.330324,2387.0,Purulia-I,24926.0,24925.0,9422.0,4904.0,42542.0,56868.0,26103.0,14060.0,4215.0,2281.0,18230.0,24726.0,11456.0,8,8,2387,B,0.0,West Bengal_Purulia_Purulia-I,1,9.375000,10.375000,1.000000
10132,2011,West Bengal,19.0,Purulia,321.0,3060.0,321106.0,163808.0,157298.0,228,72,64,46,10,14,4.0,10.0,10.0,6.0,180,12,26,50,32,100,36,0.000000,727.410598,3573.019994,1209.795310,54.592113,11.148223,385.255824,2379.0,Purulia-Ii,29324.0,29324.0,17986.0,3042.0,37084.0,58112.0,27494.0,17437.0,8037.0,1282.0,15699.0,25018.0,12581.0,9,9,2379,B,0.0,West Bengal_Purulia_Purulia-Ii,1,2.555556,3.666667,1.111111
10133,2011,West Bengal,19.0,Purulia,321.0,3061.0,193586.0,99850.0,93736.0,184,78,64,52,14,16,16.0,14.0,10.0,6.0,176,18,20,50,32,128,44,0.000000,451.627583,3047.642881,1716.875673,59.188744,37.326854,230.787399,2382.0,

In [330]:
# finalmerge2.to_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/NREGA_test_block2.csv',index=False)

## Village Level

In [373]:
gp_20 = merge1_work_match20.groupby(['state','district','Block','gp']).sum().reset_index()
gp_14 = merge1_work_match14.groupby(['state','district','Block','gp']).sum().reset_index()

print(len(gp_20))
print(len(gp_14))

241548
241713


In [374]:
# lgd = lgd_full[['state_lgd', 'dist_lgd','state', 'district', 'district_code',
#                 'block_lgd', 'Block']].drop_duplicates()

merg20 = pd.merge(gp_20,lgd_full,on=['state','district','Block','gp'],how='inner').drop_duplicates(subset=['state','district','Block','gp'])
merg14 = pd.merge(gp_14,lgd_full,on=['state','district','Block','gp'],how='inner').drop_duplicates(subset=['state','district','Block','gp'])
# merge2_work_lgd14 = pd.merge(blk_14,lgd,on=['state','district','Block'],how='inner').drop_duplicates(subset=['state','district','Block'])

print(len(merg20))
print(len(merg14))

merg20

119854
113221


,state,district,Block,gp,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,count,num,Unnamed: 0,state_lgd,dist_lgd,vil_code,Village,census2001,census2011,district_code,subdt_lgd,census2011_sdt,gpcode,block_lgd
0,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Addateegala,1001.0,1001.0,45.0,488.0,51.0,584.0,341.0,344.0,39.0,453.0,46.0,538.0,315.0,1,1,613475,28,745,586845,Dokkapalem,1786400.0,586845,4887,Addateegala,4887.0,198936.0,4865
3,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Bodlanka,370.0,370.0,11.0,337.0,62.0,410.0,222.0,239.0,11.0,330.0,61.0,402.0,219.0,1,1,613469,28,745,586853,Venkatanagaram,1787200.0,586853,4887,Addateegala,4887.0,198938.0,4865
6,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,D.Bheemavaram,951.0,951.0,12.0,987.0,78.0,1077.0,540.0,582.0,10.0,790.0,66.0,866.0,438.0,1,1,613479,28,745,586834,Nimmalapalem,1785300.0,586834,4887,Addateegala,4887.0,198939.0,4865
12,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,D.Ramavaram,555.0,555.0,26.0,783.0,26.0,835.0,416.0,414.0,20.0,638.0,11.0,669.0,344.0,1,1,613487,28,745,586814,Yellapuram,1783300.0,586814,4887,Addateegala,4887.0,198940.0,4865
19,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Dakodu,480.0,480.0,3.0,561.0,23.0,587.0,309.0,348.0,3.0,513.0,21.0,537.0,283.0,1,1,613493,28,745,586803,Jajipalem,1782200.0,586803,4887,Addateegala,4887.0,198941.0,4865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310710,West Bengal,Uttar Dinajpur,Raiganj,Mahipur,6514.0,6514.0,3988.0,841.0,4660.0,9489.0,4499.0,2616.0,1354.0,203.0,1397.0,2954.0,1724.0,1,1,345213,19,311,309628,Basian,347600.0,309628,2196,Raiganj,2196.0,109341.0,2886
310724,West Bengal,Uttar Dinajpur,Raiganj,Maraikura,5098.0,5098.0,3955.0,45.0,5621.0,9621.0,3983.0,2617.0,1976.0,20.0,2867.0,4863.0,1961.0,1,1,345199,19,311,309718,Maraikura,356600.0,309718,2196,Raiganj,2196.0,109342.0,2886
310735,West Bengal,Uttar Dinajpur,Raiganj,Rampur,5885.0,5885.0,4233.0,677.0,4947.0,9857.0,4620.0,1922.0,916.0,58.0,845.0,1819.0,1146.0,1,1,345193,19,311,309674,Lohanda,352200.0,309674,2196,Raiganj,2196.0,109343.0,2886
310750,West Bengal,Uttar Dinajpur,Raiganj,Serpur,5339.0,5339.0,5556.0,718.0,4787.0,11061.0,4785.0,3269.0,2316.0,304.0,2015.0,4635.0,2028.0,1,1,345182,19,311,309739,Patol,358700.0,309739,2196,Raiganj,2196.0,109344.0,2886


In [388]:
# gp_20_census = data20.groupby(['state','district','Block','gp']).sum().reset_index()
# gp_14_census = data11.groupby(['state','district','Block','gp']).sum().reset_index()


df20 = data20.copy()
df20 = df20.drop(columns={'sdtcode11','Village','stcode11', 'dtcode11', 'vilcode11','gp',
       'subdistrict', 'vilname11', 'vil_code','Block','subdt_lgd',	'Village'})

df11 = data11.copy()
df11 = df11.drop(columns={'sdtcode11','Village','stcode11', 'dtcode11', 'vilcode11','gp',
       'subdistrict', 'vilname11', 'vil_code','Block','subdt_lgd',	'Village'})

# List of columns to group by
groupby_columns = ['year','state', 'state_lgd', 'district', 'dist_lgd', 'block_lgd','gpcode']

# Columns to mean
mean_columns = ['SubDistrict Area','nearest_town_distance']

# Columns to sum (all columns except the groupby columns and the mean columns)
sum_columns = [col for col in df20.columns if col not in groupby_columns + mean_columns]

# Group by and aggregate
gp_20_census = df20.groupby(groupby_columns).agg({**{col: 'sum' for col in sum_columns},
                                                 **{col: 'mean' for col in mean_columns}}).reset_index()

gp_11_census = df11.groupby(groupby_columns).agg({**{col: 'sum' for col in sum_columns},
                                                 **{col: 'mean' for col in mean_columns}}).reset_index()



print(len(gp_20_census))
print(len(gp_11_census))

gp_20_census

232807
210909


,year,state,state_lgd,district,dist_lgd,block_lgd,gpcode,population,male_pop,female_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,village_area_sqkm,avg_ntl,ntl_pa,sc_pop_share,st_pop_share,SubDistrict Area,nearest_town_distance
0,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4865.0,198936.0,7216.0,3674.0,3542.0,2,1,1,1,1,1,1,1,1,1,3,1,1,2,0,3,1,10.128090,51.421112,14.961087,0,0,547.778669,0.263263
1,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4865.0,198937.0,1637.0,861.0,757.0,5,1,0,0,0,0,2,0,0,1,5,0,1,1,0,3,0,47.331388,115.215556,19.196064,0,0,547.778669,0.263263
2,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4865.0,198938.0,878.0,416.0,462.0,1,0,0,0,0,0,0,0,3,0,2,0,1,0,0,2,0,6.959429,26.152222,12.807931,0,0,547.778669,7.326928
3,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4865.0,198939.0,2158.0,1070.0,1088.0,3,0,1,0,0,0,0,0,2,1,3,0,1,0,0,2,0,224.286430,443.253330,7.391456,0,0,547.778669,1.914872
4,2020,Andhra Pradesh,28.0,Alluri Sitharama Raju,745.0,4865.0,198941.0,153.0,74.0,79.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2.149225,6.901111,3.210976,0,0,547.778669,0.263263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232802,2020,West Bengal,19.0,Purulia,321.0,3063.0,111117.0,14447.0,7271.0,7176.0,13,1,7,3,0,0,0,0,0,0,13,2,2,4,0,12,0,41.643200,197.197775,72.538894,0,0,216.977568,3.728405
232803,2020,West Bengal,19.0,Purulia,321.0,3063.0,111118.0,22344.0,11577.0,10767.0,9,1,1,1,0,1,1,0,3,1,10,0,1,1,1,9,1,23.343148,235.820001,109.511054,0,0,216.977568,5.055197
232804,2020,West Bengal,19.0,Purulia,321.0,3063.0,111119.0,13118.0,6783.0,6335.0,9,2,4,2,0,0,0,0,4,2,11,2,2,5,2,12,1,14.269177,161.204445,182.847546,0,0,216.977568,8.716370
232805,2020,West Bengal,19.0,Purulia,321.0,3063.0,111120.0,11735.0,5950.0,5785.0,14,4,3,3,0,0,2,1,3,1,14,2,2,3,0,15,0,41.331847,227.697778,95.035557,0,0,216.977568,1.828094


In [392]:
merge_vil20 = pd.merge(merg20,gp_20_census,on=['gpcode'],how='inner')
merge_vil11 = pd.merge(merg14,gp_11_census,on=['gpcode'],how='inner')

merge_vil20 = merge_vil20.drop(columns={'state_y', 'district_y',
                                                  'state_lgd_y', 'dist_lgd_y','Unnamed: 0','block_lgd_y'})
merge_vil20 = merge_vil20.rename(columns={'state_x':'state', 'district_x':'district',
                                                  'state_lgd_x':'state_lgd',
                                                  'dist_lgd_x':'dist_lgd',
                                                  'block_lgd_x':'block_lgd'})

merge_vil11 = merge_vil11.drop(columns={'state_y', 'district_y',
                                                  'state_lgd_y', 'dist_lgd_y','Unnamed: 0','block_lgd_y'})
merge_vil11 = merge_vil11.rename(columns={'state_x':'state', 'district_x':'district',
                                                  'state_lgd_x':'state_lgd',
                                                  'dist_lgd_x':'dist_lgd',
                                                  'block_lgd_x':'block_lgd'})


print(len(merge_vil20))
print(len(merge_vil11))

114859
97323


In [394]:
finalmerge_village = pd.concat([merge_vil20,merge_vil11])

# Your state_to_zone dictionary
state_to_zone = {
    'Andhra Pradesh': 'A',
    'Assam': 'B',
    'Bihar': 'B',
    'Chhattisgarh': 'B',
    'Goa': 'A',
    'Gujarat': 'A',
    'Haryana': 'B',
    'Jharkhand': 'B',
    'Karnataka': 'A',
    'Kerala': 'A',
    'Madhya Pradesh': 'B',
    'Maharashtra': 'A',
    'Odisha': 'B',
    'Punjab': 'B',
    'Rajasthan': 'B',
    'Tamil Nadu': 'A',
    'Telangana': 'A',
    'Uttar Pradesh': 'B',
    'Uttarakhand': 'B',
    'West Bengal': 'B'
}

# Filter the DataFrame
finalmerge_village['Region'] = finalmerge_village['state'].map(state_to_zone)

# Create the 'period' column based on the 'year' column
finalmerge_village['period'] = np.where(finalmerge_village['year'] == 2011, 0, 
                                    np.where(finalmerge_village['year'] == 2020, 1, np.nan))


finalmerge_village['unique_id'] = finalmerge_village['state'] + '_' + finalmerge_village['district'] + '_' + finalmerge_village['Block']+ '_' + finalmerge_village['gp']

# If you want to drop rows where 'year' is not 2011 or 2020
finalmerge_village.dropna(subset=['period'], inplace=True)


In [396]:
finalmerge_village = finalmerge_village.drop_duplicates(subset=['year','gpcode'])
finalmerge_village

,state,district,Block,gp,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,count,num,state_lgd,dist_lgd,vil_code,Village,census2001,census2011,district_code,subdt_lgd,census2011_sdt,gpcode,block_lgd,year,population,male_pop,female_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,village_area_sqkm,avg_ntl,ntl_pa,sc_pop_share,st_pop_share,SubDistrict Area,nearest_town_distance,Region,period,unique_id
0,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Addateegala,1001.0,1001.0,45.0,488.0,51.0,584.0,341.0,344.0,39.0,453.0,46.0,538.0,315.0,1,1,28,745,586845,Dokkapalem,1786400.0,586845,4887,Addateegala,4887.0,198936.0,4865,2020,7216.0,3674.0,3542.0,2,1,1,1,1,1,1.0,1.0,1.0,1.0,3,1,1,2,0,3,1,10.128090,51.421112,14.961087,0,0,547.778669,0.263263,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...
1,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Bodlanka,370.0,370.0,11.0,337.0,62.0,410.0,222.0,239.0,11.0,330.0,61.0,402.0,219.0,1,1,28,745,586853,Venkatanagaram,1787200.0,586853,4887,Addateegala,4887.0,198938.0,4865,2020,878.0,416.0,462.0,1,0,0,0,0,0,0.0,0.0,3.0,0.0,2,0,1,0,0,2,0,6.959429,26.152222,12.807931,0,0,547.778669,7.326928,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...
2,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,D.Bheemavaram,951.0,951.0,12.0,987.0,78.0,1077.0,540.0,582.0,10.0,790.0,66.0,866.0,438.0,1,1,28,745,586834,Nimmalapalem,1785300.0,586834,4887,Addateegala,4887.0,198939.0,4865,2020,2158.0,1070.0,1088.0,3,0,1,0,0,0,0.0,0.0,2.0,1.0,3,0,1,0,0,2,0,224.286430,443.253330,7.391456,0,0,547.778669,1.914872,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...
3,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Dakodu,480.0,480.0,3.0,561.0,23.0,587.0,309.0,348.0,3.0,513.0,21.0,537.0,283.0,1,1,28,745,586803,Jajipalem,1782200.0,586803,4887,Addateegala,4887.0,198941.0,4865,2020,153.0,74.0,79.0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0,0,1,0,2.149225,6.901111,3.210976,0,0,547.778669,0.263263,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...
4,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Dhanyampalem,861.0,861.0,1.0,1234.0,33.0,1268.0,647.0,614.0,1.0,1103.0,28.0,1132.0,581.0,1,1,28,745,586820,Chaparatipalem,1783900.0,586820,4887,Addateegala,4887.0,198942.0,4865,2020,344.0,166.0,178.0,1,0,0,0,0,0,0.0,0.0,2.0,0.0,1,0,0,0,0,0,0,8.700885,31.970000,7.214727,0,0,547.778669,4.903183,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97318,West Bengal,Uttar Dinajpur,Raiganj,Mahipur,6124.0,6123.0,3988.0,841.0,4660.0,9489.0,4499.0,2908.0,1354.0,203.0,1397.0,2954.0,1724.0,1,1,19,311,309628,Basian,347600.0,309628,2196,Raiganj,2196.0,109341.0,2886,2011,26528.0,13694.0,12834.0,13,3,1,1,0,0,0.0,0.0,0.0,0.0,8,1,3,3,0,6,3,46.990631,215.089766,71.609731,6.968684,3.353725,638.336107,NaN,B,0.0,West Bengal_Uttar Dinajpur_Raiganj_Mahipur
97319,West Bengal,Uttar Dinajpur,Raiganj,Maraikura,4766.0,4766.0,3955.0,45.0,5621.0,9621.0,3983.0,1389.0,1976.0,20.0,2867.0,4863.0,1961.0,1,1,19,311,309718,Maraikura,356600.0,309718,2196,Raiganj,2196.0,109342.0,2886,2011,15446.0,7967.0,7479.0,8,1,0,0,0,0,0.0,4.0,0.0,0.0,9,0,1,0,0,9,0,14.697007,157.127379,122.783377,4.451612,0.049958,638.336107,NaN,B,0.0,West Bengal_Uttar Dinajpur_Raiganj_Maraikura
97320,West Bengal,Uttar Dinajpur,Raiganj,Rampur,5542.0,5542.0,4233.0,677.0,4947.0,9857.0,4620.0,1568.0,916.0,58.0,845.0,1819.0,1146.0,1,1,19,311,309674,Lohanda,352200.0,309674,2196,Raiganj,2196.0,1

In [399]:


pivot_df = finalmerge_village.pivot(index=['gpcode'], columns='year', values=['mcwc']).reset_index()

# Rename columns for clarity
pivot_df.columns = ['gpcode', 'mcwc_2020','mcwc_2011']

# Create the 'treated' column based on the comparison of mcwc values
pivot_df['treated'] = np.where(pivot_df['mcwc_2011'] > pivot_df['mcwc_2020'], 0, 1)
# pivot_df['mcwc_dens_2011']=pivot_df['mcwc_2011']/pivot_df['num_2011']
# pivot_df['mcwc_dens_2020']=pivot_df['mcwc_2020']/pivot_df['num_2020']
# pivot_df['intensity']=pivot_df['mcwc_dens_2020']-pivot_df['mcwc_dens_2011']

# Adjust intensity based on the conditions
# pivot_df['intensity'] = pivot_df.apply(
#     lambda row: 0 if row['intensity'] < 0 or row['mcwc_2011'] == row['mcwc_2020'] else row['intensity'], 
#     axis=1
# )



# Merge the treated column back to the original DataFrame
finalmerge2_vill = pd.merge(finalmerge_village, pivot_df[['gpcode', 'treated',
                                            #  'intensity','mcwc_dens_2020','mcwc_dens_2011'
                                             ]]
                                , on=['gpcode'], how='left')

# Drop duplicates to avoid issues in the merge step
finalmerge2_vill = finalmerge2_vill.drop_duplicates(subset=['gpcode', 'year'])

finalmerge2_vill


,state,district,Block,gp,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,count,num,state_lgd,dist_lgd,vil_code,Village,census2001,census2011,district_code,subdt_lgd,census2011_sdt,gpcode,block_lgd,year,population,male_pop,female_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,village_area_sqkm,avg_ntl,ntl_pa,sc_pop_share,st_pop_share,SubDistrict Area,nearest_town_distance,Region,period,unique_id,treated
0,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Addateegala,1001.0,1001.0,45.0,488.0,51.0,584.0,341.0,344.0,39.0,453.0,46.0,538.0,315.0,1,1,28,745,586845,Dokkapalem,1786400.0,586845,4887,Addateegala,4887.0,198936.0,4865,2020,7216.0,3674.0,3542.0,2,1,1,1,1,1,1.0,1.0,1.0,1.0,3,1,1,2,0,3,1,10.128090,51.421112,14.961087,0,0,547.778669,0.263263,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...,0
1,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Bodlanka,370.0,370.0,11.0,337.0,62.0,410.0,222.0,239.0,11.0,330.0,61.0,402.0,219.0,1,1,28,745,586853,Venkatanagaram,1787200.0,586853,4887,Addateegala,4887.0,198938.0,4865,2020,878.0,416.0,462.0,1,0,0,0,0,0,0.0,0.0,3.0,0.0,2,0,1,0,0,2,0,6.959429,26.152222,12.807931,0,0,547.778669,7.326928,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...,0
2,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,D.Bheemavaram,951.0,951.0,12.0,987.0,78.0,1077.0,540.0,582.0,10.0,790.0,66.0,866.0,438.0,1,1,28,745,586834,Nimmalapalem,1785300.0,586834,4887,Addateegala,4887.0,198939.0,4865,2020,2158.0,1070.0,1088.0,3,0,1,0,0,0,0.0,0.0,2.0,1.0,3,0,1,0,0,2,0,224.286430,443.253330,7.391456,0,0,547.778669,1.914872,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...,0
3,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Dakodu,480.0,480.0,3.0,561.0,23.0,587.0,309.0,348.0,3.0,513.0,21.0,537.0,283.0,1,1,28,745,586803,Jajipalem,1782200.0,586803,4887,Addateegala,4887.0,198941.0,4865,2020,153.0,74.0,79.0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0,0,1,0,2.149225,6.901111,3.210976,0,0,547.778669,0.263263,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...,1
4,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,Dhanyampalem,861.0,861.0,1.0,1234.0,33.0,1268.0,647.0,614.0,1.0,1103.0,28.0,1132.0,581.0,1,1,28,745,586820,Chaparatipalem,1783900.0,586820,4887,Addateegala,4887.0,198942.0,4865,2020,344.0,166.0,178.0,1,0,0,0,0,0,0.0,0.0,2.0,0.0,1,0,0,0,0,0,0,8.700885,31.970000,7.214727,0,0,547.778669,4.903183,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateega...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206134,West Bengal,Uttar Dinajpur,Raiganj,Mahipur,6124.0,6123.0,3988.0,841.0,4660.0,9489.0,4499.0,2908.0,1354.0,203.0,1397.0,2954.0,1724.0,1,1,19,311,309628,Basian,347600.0,309628,2196,Raiganj,2196.0,109341.0,2886,2011,26528.0,13694.0,12834.0,13,3,1,1,0,0,0.0,0.0,0.0,0.0,8,1,3,3,0,6,3,46.990631,215.089766,71.609731,6.968684,3.353725,638.336107,NaN,B,0.0,West Bengal_Uttar Dinajpur_Raiganj_Mahipur,0
206135,West Bengal,Uttar Dinajpur,Raiganj,Maraikura,4766.0,4766.0,3955.0,45.0,5621.0,9621.0,3983.0,1389.0,1976.0,20.0,2867.0,4863.0,1961.0,1,1,19,311,309718,Maraikura,356600.0,309718,2196,Raiganj,2196.0,109342.0,2886,2011,15446.0,7967.0,7479.0,8,1,0,0,0,0,0.0,4.0,0.0,0.0,9,0,1,0,0,9,0,14.697007,157.127379,122.783377,4.451612,0.049958,638.336107,NaN,B,0.0,West Bengal_Uttar Dinajpur_Raiganj_Maraikura,0
206136,West Bengal,Uttar Dinajpur,Raiganj,Rampur,5542.0,5542.0,4233.0,677.0,4947.0,9857.0,4620.0,1568.0,916.0,58.0,845.0,1819.0,1146.0,1,1,19,311,309674,Lohanda,352200.0

In [401]:
# finalmerge2_vill.to_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/NREGA_test_gp.csv',index=False)

## Misc

In [410]:
test3 = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/NREGA_test_block.csv')


In [418]:
test3

,state,district,Block,State Code,District Code,Subdistrict Code,subdistrict,Development Block Code,year,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,population,male_pop,female_pop,sc_pop,st_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,mcwc,dispensary,veterinary_hospital,aanganwadi,post_office,bus,railway,all_weather_road,cooperative_bank,mandis,nearest_town_distance,village_area_sqkm,SubDistrict Area,avg_ntl,dist_lgd,state_lgd,subdt_lgd,block_lgd,Region,period,unique_id,treated
0,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,28.0,745.0,4887.0,Addateegala,4865.0,2019,12545.0,12545.0,947.0,11962.0,2282.0,15191.0,7743.0,8413.0,827.0,10771.0,1874.0,13472.0,6893.0,31713.0,15893.0,15801.0,0.0,0.0,45,9,6,2,1,2,5.0,21.0,2.0,5.0,47,15,16,0,30,1,2,2.804197,485.686671,547.778669,1265.577775,745.0,28.0,4887.0,4865.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateegala,1
1,Andhra Pradesh,Alluri Sitharama Raju,Ananthagiri,28.0,745.0,4845.0,Ananthagiri,5584.0,2019,12986.0,12986.0,430.0,24946.0,339.0,25715.0,12652.0,10828.0,399.0,23699.0,308.0,24406.0,11953.0,36264.0,18263.0,17876.0,0.0,0.0,86,13,11,5,0,1,14.0,43.0,5.0,18.0,132,24,24,3,75,5,5,3.616160,322.971890,923.270600,1184.831524,745.0,28.0,4845.0,5584.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Ananthagiri,1
2,Andhra Pradesh,Alluri Sitharama Raju,Araku Valley,28.0,745.0,4844.0,Araku Valley,5585.0,2019,15844.0,15844.0,136.0,28585.0,159.0,28880.0,14649.0,13114.0,118.0,27074.0,144.0,27336.0,13809.0,57376.0,26087.0,27250.0,0.0,0.0,90,11,8,5,1,1,14.0,20.0,1.0,9.0,100,5,8,3,54,2,5,2.884879,206.193145,367.275295,1062.759993,745.0,28.0,4844.0,5585.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Araku Valley,1
3,Andhra Pradesh,Alluri Sitharama Raju,Chintapalle,28.0,745.0,4849.0,Chinthapalli,5589.0,2019,15715.0,15715.0,206.0,31153.0,282.0,31641.0,15811.0,13178.0,177.0,28031.0,206.0,28414.0,14069.0,61511.0,29883.0,31235.0,0.0,0.0,107,22,6,3,5,6,39.0,35.0,4.0,18.0,126,35,19,0,71,3,3,8.820650,392.219822,679.280761,1542.236657,745.0,28.0,4849.0,5589.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Chintapalle,1
4,Andhra Pradesh,Alluri Sitharama Raju,Chintoor,28.0,745.0,4732.0,Chinturu,5062.0,2019,11875.0,11875.0,226.0,15158.0,683.0,16067.0,8500.0,7899.0,163.0,13311.0,467.0,13941.0,7375.0,41063.0,20395.0,20365.0,0.0,0.0,71,13,8,6,1,0,8.0,28.0,10.0,13.0,76,13,11,1,37,7,2,5.829834,668.703762,1153.987909,1784.150000,745.0,28.0,4732.0,5062.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Chintoor,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10645,West Bengal,Uttar Dinajpur,Islampur,19.0,311.0,2192.0,Islampur,2882.0,2014,50852.0,50760.0,15219.0,1543.0,94811.0,111573.0,52733.0,17162.0,5749.0,505.0,43225.0,49479.0,25048.0,317675.0,164315.0,153360.0,55038.0,7733.0,106,42,19,10,0,0,1.0,5.0,1.0,1.0,62,3,8,3,70,8,17,NaN,404.102577,438.760141,1719.156055,311.0,19.0,2192.0,2882.0,B,0.0,West Bengal_Uttar Dinajpur_Islampur,1
10646,West Bengal,Uttar Dinajpur,Itahar,19.0,311.0,2199.0,Itahar,2883.0,2014,52494.0,52491.0,27937.0,5971.0,63999.0,97907.0,45325.0,10874.0,11136.0,1844.0,32383.0,45363.0,22012.0,289242.0,148403.0,140839.0,74733.0,23876.0,186,42,19,14,3,4,4.0,27.0,20.0,8.0,161,6,35,1,136,4,30,NaN,446.631741,449.882572,1628.630952,311.0,19.0,2199.0,2883.0,B,0.0,West Bengal_Uttar Dinajpur_Itahar,0
10647,West Bengal,Uttar Dinajpur,Kaliaganj,19.0,311.0,2198.0,Kaliaganj,2884.0,2014,51120.0,51079.0,65727.0,4984.0,30295.0,101006.0,46568.0,34149.0,20303.0,1206.0,8162.0,29671.0,16363.0,223919.0,114979.0,108940.0,138238.0,12675.0,159,34,20,20,1,2,3.0,0.0,4.0,5.0,159,2,22,3,59,9,40,NaN,

In [415]:
test3a = test3[test3['year']==2019]
test3b = test3[test3['year']==2014]
print(len(test3a))
print(len(test3b))
df20_agg_filt = df20_agg[['state_lgd','dist_lgd','block_lgd','count']]
df11_agg_filt = df11_agg[['state_lgd','dist_lgd','block_lgd','count']]

test4a = pd.merge(df20_agg_filt,test3a, on=['state_lgd','dist_lgd','block_lgd'],how='inner')
test4b = pd.merge(df11_agg_filt,test3b, on=['state_lgd','dist_lgd','block_lgd'],how='inner')
print(len(test4a))
print(len(test4b))

5732
4918
5732
4918


In [420]:
finalmerge

,state_lgd,dist_lgd,block_lgd,count,state,district,Block,State Code,District Code,Subdistrict Code,subdistrict,Development Block Code,year,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,population,male_pop,female_pop,sc_pop,st_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,mcwc,dispensary,veterinary_hospital,aanganwadi,post_office,bus,railway,all_weather_road,cooperative_bank,mandis,nearest_town_distance,village_area_sqkm,SubDistrict Area,avg_ntl,subdt_lgd,Region,period,unique_id,treated
0,28.0,745.0,4865.0,56,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,28.0,745.0,4887.0,Addateegala,4865.0,2019,12545.0,12545.0,947.0,11962.0,2282.0,15191.0,7743.0,8413.0,827.0,10771.0,1874.0,13472.0,6893.0,31713.0,15893.0,15801.0,0.0,0.0,45,9,6,2,1,2,5.0,21.0,2.0,5.0,47,15,16,0,30,1,2,2.804197,485.686671,547.778669,1265.577775,4887.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateegala,1
1,28.0,745.0,4874.0,36,Andhra Pradesh,Alluri Sitharama Raju,Devipatnam,28.0,745.0,4885.0,Devipatnam,4874.0,2019,9314.0,9314.0,708.0,8234.0,2502.0,11444.0,5620.0,6366.0,606.0,7292.0,2042.0,9940.0,4868.0,27947.0,13215.0,14687.0,0.0,0.0,32,8,5,2,1,0,6.0,15.0,0.0,2.0,33,15,9,0,19,1,1,5.348022,331.333283,377.227968,1075.919998,4885.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Devipatnam,1
2,28.0,745.0,4876.0,36,Andhra Pradesh,Alluri Sitharama Raju,Gangavaram,28.0,745.0,4894.0,Gangavaram,4876.0,2019,10455.0,10455.0,1058.0,10001.0,3617.0,14676.0,7745.0,7339.0,978.0,9279.0,3379.0,13636.0,7248.0,23219.0,11002.0,12115.0,0.0,0.0,28,11,5,3,0,1,13.0,18.0,1.0,7.0,30,13,8,0,22,5,1,5.604924,352.122052,378.724208,913.918889,4894.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Gangavaram,1
3,28.0,745.0,4895.0,25,Andhra Pradesh,Alluri Sitharama Raju,Maredumilli,28.0,745.0,4884.0,Maredumilli,4895.0,2019,5988.0,5988.0,39.0,7513.0,45.0,7597.0,3871.0,3983.0,22.0,6309.0,33.0,6364.0,3261.0,10846.0,5463.0,5382.0,0.0,0.0,19,5,2,2,0,1,4.0,7.0,0.0,3.0,20,3,4,0,8,1,1,1.579791,91.935201,1079.650769,264.876667,4884.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Maredumilli,1
4,28.0,745.0,4905.0,55,Andhra Pradesh,Alluri Sitharama Raju,Rajavommangi,28.0,745.0,4888.0,Rajavommangi,4905.0,2019,12551.0,12551.0,285.0,11182.0,5317.0,16784.0,8657.0,8945.0,256.0,10230.0,4991.0,15477.0,8023.0,42730.0,21429.0,21191.0,0.0,0.0,50,14,8,3,3,2,7.0,22.0,7.0,6.0,52,17,10,0,39,3,3,4.653360,297.879966,528.940158,959.093192,4888.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Rajavommangi,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,19.0,321.0,3059.0,222,West Bengal,Purulia,PuruliaI,19.0,321.0,2387.0,Purulia - I,3059.0,2014,24926.0,24925.0,9422.0,4904.0,42542.0,56868.0,26103.0,14060.0,4215.0,2281.0,18230.0,24726.0,11456.0,285238.0,146748.0,138490.0,46004.0,24776.0,218,76,60,36,22,24,12.0,8.0,8.0,16.0,196,50,20,28,108,22,28,NaN,662.198467,338.330324,2389.560003,2387.0,B,0.0,West Bengal_Purulia_PuruliaI,1
4914,19.0,321.0,3060.0,234,West Bengal,Purulia,PuruliaIi,19.0,321.0,2379.0,Purulia - II,3060.0,2014,29324.0,29324.0,17986.0,3042.0,37084.0,58112.0,27494.0,17437.0,8037.0,1282.0,15699.0,25018.0,12581.0,321106.0,163808.0,157298.0,84710.0,16240.0,228,72,64,46,10,14,4.0,10.0,10.0,6.0,180,26,50,32,100,12,36,NaN,727.410598,385.255824,3573.019994,2379.0,B,0.0,West Bengal_Purulia_PuruliaIi,1
4915,19.0,321.0,3061.0,198,West Bengal,Purulia,Raghunath PurI,19.0,321.0,2382.0,Raghunathpur - I,3061.0,2014,20272.0,20272.0,21912.0,5813.0,21132.0,48857.0,22278.0,11254.0,11185.0,3312.0,9137.0,23634.0,10563.0,193586.0,99850.0,93736.0,77026.0,23996.0,184,78,64,52,14,16,16.0,10.0,14.0,6.0,176

In [421]:
finalmerge = pd.concat([test4a,test4b])



pivot_df = finalmerge.pivot(index=['state','district','block_lgd'], columns='year', values=['mcwc','count']).reset_index()

# Rename columns for clarity
pivot_df.columns = ['state','district','block_lgd', 'mcwc_2011', 'mcwc_2020','num_2011','num_2020']

# Create the 'treated' column based on the comparison of mcwc values
# pivot_df['treated'] = np.where(pivot_df['mcwc_2011'] > pivot_df['mcwc_2020'], 0, 1)
pivot_df['mcwc_dens_2011']=pivot_df['mcwc_2011']/pivot_df['num_2011']
pivot_df['mcwc_dens_2020']=pivot_df['mcwc_2020']/pivot_df['num_2020']
pivot_df['intensity']=pivot_df['mcwc_dens_2020']-pivot_df['mcwc_dens_2011']

# Adjust intensity based on the conditions
pivot_df['intensity'] = pivot_df.apply(
    lambda row: 0 if row['intensity'] < 0 or row['mcwc_2011'] == row['mcwc_2020'] else row['intensity'], 
    axis=1
)



# Merge the treated column back to the original DataFrame
finalmerge2 = pd.merge(finalmerge, pivot_df[['state','district','block_lgd', #'treated',
                                             'intensity','mcwc_dens_2020','mcwc_dens_2011'
                                             ]]
                                , on=['state','district','block_lgd'], how='left')

# Drop duplicates to avoid issues in the merge step
finalmerge2 = finalmerge2.drop_duplicates(subset=['block_lgd', 'year'])

finalmerge2


,state_lgd,dist_lgd,block_lgd,count,state,district,Block,State Code,District Code,Subdistrict Code,subdistrict,Development Block Code,year,applied_jobcards,issued_jobcards,registered_workers_scs,registered_workers_sts,registered_workers_others,total_registered_workers,registered_workers_women,active_jobcards,active_workers_scs,active_workers_sts,active_workers_others,total_active_workers,active_workers_women,population,male_pop,female_pop,sc_pop,st_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,mcwc,dispensary,veterinary_hospital,aanganwadi,post_office,bus,railway,all_weather_road,cooperative_bank,mandis,nearest_town_distance,village_area_sqkm,SubDistrict Area,avg_ntl,subdt_lgd,Region,period,unique_id,treated,intensity,mcwc_dens_2020,mcwc_dens_2011
0,28.0,745.0,4865.0,56,Andhra Pradesh,Alluri Sitharama Raju,Addateegala,28.0,745.0,4887.0,Addateegala,4865.0,2019,12545.0,12545.0,947.0,11962.0,2282.0,15191.0,7743.0,8413.0,827.0,10771.0,1874.0,13472.0,6893.0,31713.0,15893.0,15801.0,0.0,0.0,45,9,6,2,1,2,5.0,21.0,2.0,5.0,47,15,16,0,30,1,2,2.804197,485.686671,547.778669,1265.577775,4887.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Addateegala,1,0.375000,0.375000,0.000000
1,28.0,745.0,4874.0,36,Andhra Pradesh,Alluri Sitharama Raju,Devipatnam,28.0,745.0,4885.0,Devipatnam,4874.0,2019,9314.0,9314.0,708.0,8234.0,2502.0,11444.0,5620.0,6366.0,606.0,7292.0,2042.0,9940.0,4868.0,27947.0,13215.0,14687.0,0.0,0.0,32,8,5,2,1,0,6.0,15.0,0.0,2.0,33,15,9,0,19,1,1,5.348022,331.333283,377.227968,1075.919998,4885.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Devipatnam,1,0.416667,0.416667,0.000000
2,28.0,745.0,4876.0,36,Andhra Pradesh,Alluri Sitharama Raju,Gangavaram,28.0,745.0,4894.0,Gangavaram,4876.0,2019,10455.0,10455.0,1058.0,10001.0,3617.0,14676.0,7745.0,7339.0,978.0,9279.0,3379.0,13636.0,7248.0,23219.0,11002.0,12115.0,0.0,0.0,28,11,5,3,0,1,13.0,18.0,1.0,7.0,30,13,8,0,22,5,1,5.604924,352.122052,378.724208,913.918889,4894.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Gangavaram,1,0.500000,0.500000,0.000000
3,28.0,745.0,4895.0,25,Andhra Pradesh,Alluri Sitharama Raju,Maredumilli,28.0,745.0,4884.0,Maredumilli,4895.0,2019,5988.0,5988.0,39.0,7513.0,45.0,7597.0,3871.0,3983.0,22.0,6309.0,33.0,6364.0,3261.0,10846.0,5463.0,5382.0,0.0,0.0,19,5,2,2,0,1,4.0,7.0,0.0,3.0,20,3,4,0,8,1,1,1.579791,91.935201,1079.650769,264.876667,4884.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Maredumilli,1,0.280000,0.280000,0.000000
4,28.0,745.0,4905.0,55,Andhra Pradesh,Alluri Sitharama Raju,Rajavommangi,28.0,745.0,4888.0,Rajavommangi,4905.0,2019,12551.0,12551.0,285.0,11182.0,5317.0,16784.0,8657.0,8945.0,256.0,10230.0,4991.0,15477.0,8023.0,42730.0,21429.0,21191.0,0.0,0.0,50,14,8,3,3,2,7.0,22.0,7.0,6.0,52,17,10,0,39,3,3,4.653360,297.879966,528.940158,959.093192,4888.0,A,1.0,Andhra Pradesh_Alluri Sitharama Raju_Rajavommangi,1,0.400000,0.400000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10645,19.0,321.0,3059.0,222,West Bengal,Purulia,PuruliaI,19.0,321.0,2387.0,Purulia - I,3059.0,2014,24926.0,24925.0,9422.0,4904.0,42542.0,56868.0,26103.0,14060.0,4215.0,2281.0,18230.0,24726.0,11456.0,285238.0,146748.0,138490.0,46004.0,24776.0,218,76,60,36,22,24,12.0,8.0,8.0,16.0,196,50,20,28,108,22,28,NaN,662.198467,338.330324,2389.560003,2387.0,B,0.0,West Bengal_Purulia_PuruliaI,1,0.790887,0.826923,0.036036
10646,19.0,321.0,3060.0,234,West Bengal,Purulia,PuruliaIi,19.0,321.0,2379.0,Purulia - II,3060.0,2014,29324.0,29324.0,17986.0,3042.0,37084.0,58112.0,27494.0,17437.0,8037.0,1282.0,15699.0,25018.0,12581.0,321106.0,163808.0,157298.0,84710.0,16240.0,228,72,64,46,10,14,4.0,10.0,10.0,6.0,180,26,50,32,100,12,36,NaN,727.410598,385.255824,3573.019994,2379.0,B,0.0,West Bengal_Purulia_PuruliaIi,1,0.234051,0.276786,0.042735
10647,19.0,321.0,3061.0,198,West Bengal,Purulia,R

In [422]:
# finalmerge2.to_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/NREGA_test_block3.csv',index=False)